### IMPORT DATASETS AND LIBRARIES

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Accuracy

from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report,accuracy_score,roc_curve,confusion_matrix


In [ ]:
# Load the training dataset                              # Reference: https://raw.githubusercontent.com/DURGESH716/Fake-Instagram-Profile-Detection/main/insta_train.csv
instagram_df_train=pd.read_csv("insta_train.csv")        # Licence : MIT
instagram_df_train


In [ ]:
# Load the testing data                              # Reference: https://raw.githubusercontent.com/DURGESH716/Fake-Instagram-Profile-Detection/main/insta_test.csv
instagram_df_test=pd.read_csv("insta_test.csv")      # Licence : MIT
instagram_df_test


In [ ]:
instagram_df_train.head(5)

In [ ]:
instagram_df_train.tail(5)

In [ ]:
instagram_df_test.head(5)

In [ ]:
instagram_df_test.tail(5)

### PERFORM EXPLORATORY DATA ANALYSIS

In [ ]:
# Getting dataframe info
instagram_df_train.info()

In [ ]:
# Get the statistical summary of the dataframe
instagram_df_train.describe()

In [ ]:
# Checking if null values exist
instagram_df_train.isnull().sum()


In [ ]:
# Get the number of unique values in the "profile pic" feature
instagram_df_train['profile pic'].value_counts()

In [ ]:
# Get the number of unique values in "fake" (Target column)
instagram_df_train['fake'].value_counts()


In [ ]:
instagram_df_test.info()

In [ ]:
instagram_df_test.describe()

In [ ]:
instagram_df_test.isnull().sum()

In [ ]:
instagram_df_test['profile pic'].value_counts()

In [ ]:
instagram_df_test['fake'].value_counts()

### PERFORM DATA VISUALIZATION

In [ ]:
# Visualize the data
sns.countplot(instagram_df_train['fake'])

In [ ]:
# Visualize the private column data
sns.countplot(instagram_df_train['private'])

In [ ]:
# Visualize the "profile pic" column data
sns.countplot(instagram_df_train['profile pic'])

In [ ]:
# Visualize the data
plt.figure(figsize = (20, 10))
sns.distplot(instagram_df_train['nums/length username'])


In [ ]:
# Plot pairplot
plt.figure(figsize = (20, 20))
sns.pairplot(instagram_df_train)

In [ ]:
# Correlation plot
plt.figure(figsize=(20, 20))
cm = instagram_df_train.corr()
ax = plt.subplot()
sns.heatmap(cm, annot = True, ax = ax)

In [ ]:
sns.countplot(instagram_df_test['fake'])

In [ ]:
sns.countplot(instagram_df_test['profile pic'])

In [ ]:
sns.countplot(instagram_df_test['private'])

### PREPARE THE DATA TO FEED THE MODEL

In [ ]:
# Training and testing dataset (inputs)
X_train=instagram_df_train.drop(columns=['fake'])
X_test=instagram_df_test.drop(columns=['fake'])
X_train

In [ ]:
X_test

In [ ]:
# Training and testing dataset (Outputs)
y_train = instagram_df_train['fake']
y_test = instagram_df_test['fake']

In [ ]:
y_train

In [ ]:
y_test

In [ ]:
# Scale the data before training the model
from sklearn.preprocessing import StandardScaler, MinMaxScaler

scaler_x=StandardScaler()
X_train=scaler_x.fit_transform(X_train)
X_test=scaler_x.fit_transform(X_test)

In [ ]:

y_train = tf.keras.utils.to_categorical(y_train, num_classes = 2)
y_test = tf.keras.utils.to_categorical(y_test, num_classes = 2)

In [ ]:
y_train

In [ ]:
y_test

In [ ]:
X_train.shape,X_test.shape

In [ ]:
y_train.shape,y_test.shape

In [ ]:
print(len(X_train)/(len(X_train)+len(X_test))*100)

### BUILD AND TRAIN A SIMPLE DEEP LEARNING MODEL

In [ ]:
import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

model=Sequential()
model.add(Dense(50,input_dim=X_train.shape[1],activation="relu"))
model.add(Dense(150,activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(25,activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(2,activation="softmax"))

In [ ]:
model.summary()

In [ ]:
opt=keras.optimizers.Adam(learning_rate=0.01)
model.compile(optimizer=opt,loss="categorical_crossentropy",metrics=["accuracy"])

In [ ]:
epochs_hist=model.fit(X_train,y_train,epochs=500,verbose=1,validation_split=0.1)

### ASSESS THE PERORMANCE OF TRAINED MODEL

In [ ]:
print(epochs_hist.history.keys())

In [ ]:
plt.plot(epochs_hist.history['loss'])
plt.plot(epochs_hist.history['val_loss'])

plt.title('Model Loss Progression During Training/Validation')
plt.ylabel('Training and Validation Losses')
plt.xlabel('Epoch Number')
plt.legend(['Training Loss', 'Validation Loss'])


In [ ]:
predicted=model.predict(X_test)

In [ ]:
predicted_value = []
test = []
for i in predicted:
    predicted_value.append(np.argmax(i))
    
for i in y_test:
    test.append(np.argmax(i))

In [ ]:
print(classification_report(test,predicted_value))

In [ ]:
plt.figure(figsize=(10, 10))
cm=confusion_matrix(test,predicted_value)
sns.heatmap(cm,annot=True)